In [ ]:
import imp
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from context import rf_pool

In [ ]:
from rf_pool import modules, models
from rf_pool.utils import functions, datasets

In [ ]:
imp.reload(datasets)
imp.reload(functions)
imp.reload(modules)
imp.reload(models)

In [ ]:
# get MNIST training data
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root='../rf_pool/data', train=True, download=True, transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)

In [ ]:
# initialize model
dbn = rf_pool.models.DeepBeliefNetwork()

In [ ]:
# create receptive field pooling layer for probabilistic max pooling (Lee et al., 2009)
rf_layer = rf_pool.layers.RF_Pool(kernel_size=2, pool_type='prob')

In [ ]:
# append two RBM layers with sigmoid vis/hid activations
dbn.append(0, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(1, 24, 11), activation=None, pool=rf_layer, 
                                  vis_activation_fn=torch.nn.Sigmoid(), vis_sample_fn=torch.distributions.Bernoulli,
                                  hid_sample_fn=torch.distributions.Bernoulli))
dbn.append(1, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(24, 40, 9), activation=torch.nn.Sigmoid(), pool=None,
                                  vis_activation_fn=torch.nn.Sigmoid(), vis_sample_fn=torch.distributions.Bernoulli,
                                  hid_sample_fn=torch.distributions.Bernoulli))

In [ ]:
# train first layer with sparsity, monitor weights and negative examples
optim = torch.optim.SGD(dbn.layers['0'].parameters(), lr=2e-2, momentum=0.9, weight_decay=2e-3)
dbn.train('0', 1, trainloader, optim, monitor=100, k=1,
          show_weights={'layer_id': '0', 'cmap': 'gray'}, 
          show_negative={'k': 1, 'cmap': 'gray'},
          sparsity={'module_name': None, 'target': 0.002, 'cost': 5.})

In [ ]:
# train second layer, monitor weights and negative examples
# note: second layer weights are projected to the image space
optim = torch.optim.SGD(dbn.layers['1'].parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)
dbn.train('1', 1, trainloader, optim, monitor=100, k=1,
          show_weights={'layer_id': '1', 'cmap': 'gray'},
          show_negative={'k': 1, 'cmap': 'gray'})